In [1]:
from collections import Counter

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import scipy.stats as st
from scipy.cluster import hierarchy
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [3]:
def conditional_probability(data_frame, personal_data, split_method, threshold):
    
    """
    TEXT
    TEXT
    """
    
    if split_method == "standard":
        good_clients = data_frame[(data_frame["GOOD"] == 1) | (data_frame["SUFFICIENT"] == 1)]
        bad_clients = data_frame[(data_frame["GOOD"] == 0) & (data_frame["SUFFICIENT"] == 0)]
    elif split_method == "good":
        good_clients = data_frame[data_frame["GOOD"] == 1]
        bad_clients = data_frame[data_frame["GOOD"] == 0]
    elif split_method == "sufficient":
        data_frame = data_frame[data_frame["GOOD"] == 0]
        good_clients = data_frame[data_frame["SUFFICIENT"] == 1]
        bad_clients = data_frame[data_frame["SUFFICIENT"] == 0]
    
    for i in range(1, len(personal_data)+1):
        records = 0

        for key, value in personal_data.items():
            conditional_records = len(data_frame[data_frame[key] == value])
            if conditional_records > records:
                records = conditional_records
                specific_key = key
                specific_value = value
                
        good_records = len(good_clients[good_clients[specific_key] == specific_value])
        bad_records = len(bad_clients[bad_clients[specific_key] == specific_value])
        client_records = good_records + bad_records

        if client_records >= threshold and bad_records > 0:
            personal_data = {key:personal_data[key] for key in personal_data if key!=specific_key}
            good_clients = good_clients[good_clients[specific_key] == specific_value]
            bad_clients = bad_clients[bad_clients[specific_key] == specific_value]
            probability = round(bad_records/(bad_records + good_records), 5)
        else:
            break

            
    return probability, personal_data

In [4]:
main_features = ['OCCUPATION_TYPE',
                 'AGE',
                 'INCOME',
                 'FLAG_EMAIL',
                 'NAME_FAMILY_STATUS',
                 'NAME_EDUCATION_TYPE',
                 'NAME_INCOME_TYPE',
                 'MEMBERS']

extra_features = ['FLAG_OWN_CAR',
                  'FLAG_OWN_REALTY',
                  'NAME_HOUSING_TYPE',
                  'FLAG_WORK_PHONE',
                  'FLAG_PHONE',
                  'CHILDREN',
                  'YEARS_EMPLOYED',
                  'CODE_GENDER']

In [5]:
def executor(data_frame, personal_data, main_features, extra_features,
             split_method = "standard", threshold = 1000, second_threshold = 500):
    
    """
    TEXT
    TEXT
    """
    
    main_data = {feature: personal_data[feature] for feature in main_features}
    extra_data = {feature: personal_data[feature] for feature in extra_features}
    
    result = conditional_probability(data_frame, main_data, split_method, threshold)
    probability, data = result
    
    if len(data) != 0:
        result = conditional_probability(data_frame, main_data, split_method, second_threshold)
        new_probability, new_data = result
        
        if len(new_data) < len(data) and new_probability <= probability:
            probability = new_probability
            data = new_data
        
        extra_data = dict(extra_data, **data)    
        
    for key, value in extra_data.items():
        single_dict = {key: value}
        new_probability = conditional_probability(data_frame, dict(main_data, **single_dict),
                                                  split_method, second_threshold)[0]
        
        if new_probability <= probability:
            probability = new_probability
            main_data = dict(main_data, **single_dict)
    
    
    return probability